In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing as mp
from datetime import datetime
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.optim import Adam
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
# from src.model import *
# from src.util import *
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [2]:
trained_model_path = '/workdir/security/home/junjiehuang2468/paper/trained_models_weight/kaggle_miscrosoft/'
best_trained_model = '2022-01-18 19:10/50w_epoch:0_test_acc:0.905704.pt'
data_path = "/workdir/security/home/junjiehuang2468/paper/data/kaggle/"
train_data_path = data_path + "malwares/"  # Training data
train_label_path = data_path + "train_labels.csv"  # Training label

In [3]:
result_path = '/workdir/security/home/junjiehuang2468/paper/results/kaggle_miscrosoft/'

In [4]:
CUDA = True if torch.cuda.is_available() else False
NUM_WORKERS = 16  # Number of cores to use for data loader
BATCH_SIZE = 3  #
LEAVE_BIT_NUMBER = 500000
KERNEL_SIZE = 500  # Kernel size & stride for Malconv (defualt : 500)

In [5]:
trainset = pd.read_csv(data_path + 'train_dataset.csv')
validset = pd.read_csv(data_path + 'valid_dataset.csv')

In [6]:
class ExeDataset(Dataset):
    def __init__(self, malware_names, data_path, labels, leave_bit_num):
        self.malware_names = malware_names
        self.data_path = data_path
        self.labels = labels
        self.leave_bit_num = leave_bit_num

    def __len__(self):
        return len(self.malware_names)

    def __getitem__(self, idx):
        with open(self.data_path + self.malware_names[idx] + '.txt','rb') as fp:
            data = [bit+1 for bit in fp.read()[:self.leave_bit_num]]
            padding = [0]*(self.leave_bit_num-len(data))
            data = data + padding

        return np.array(data), np.array([self.labels[idx]])

In [7]:
train_dataset = ExeDataset(
    trainset["id"].tolist(), 
    train_data_path, 
    trainset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)
valid_dataset = ExeDataset(
    validset["id"].tolist(), 
    train_data_path, 
    validset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)

In [8]:
trainloader = DataLoader(
    dataset = train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = NUM_WORKERS,
    pin_memory = True
)
validloader = DataLoader(
    dataset = valid_dataset,
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = NUM_WORKERS,
    pin_memory = True
)

In [9]:
def mp_func(i,inpu,te,gr):
    check = 0
    grad_cum = 0
    for j,(inp,g,t) in enumerate(zip(inpu,gr,te)):
        if inp != 0: 
            check = j
            continue
        max_idx = np.argmin(g).tolist()
        org_max_idx = np.argmax(t).tolist()
        if g[max_idx] > 0: continue
        grad_cum += g[max_idx]
        te[j][org_max_idx] = 0
        te[j][max_idx] = 1
    return [i,te,check,grad_cum]

In [10]:
# class Model(nn.Module):
#     def __init__(self, data_length = 2e6, kernel_size = 500):
#         super().__init__()
#         self.embedding = nn.Embedding(257, 8, padding_idx=0)
#         self.conv_layer_1 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
#         # self.bn_1 = nn.BatchNorm1d(128)
#         self.conv_layer_2 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
#         self.pool_layer_2 = nn.MaxPool1d(data_length//kernel_size)
#         self.fc_layer_3 = nn.Linear(128, 128)
#         self.fc_layer_4 = nn.Linear(128, 9)
        
#     def forward(self,x):
#         x = self.embedding(x)
#         x = x.transpose(-1,-2)
#         x_conv_1 = self.conv_layer_1(x[:,:4,:])
#         x_conv_2 = torch.sigmoid(self.conv_layer_2(x[:,4:,:]))
#         x = x_conv_1*x_conv_2
#         del x_conv_1,x_conv_2
#         x = self.pool_layer_2(x).squeeze()
#         x = self.fc_layer_3(x)
#         x = self.fc_layer_4(x)
#         # x = torch.sigmoid(x)
#         return x

In [11]:
class Model(nn.Module):
    def __init__(self, data_length = 2e6, kernel_size = 500):
        super().__init__()
        self.embedding = nn.Embedding(257, 8, padding_idx=0)
        self.conv_layer_1 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        # self.bn_1 = nn.BatchNorm1d(128)
        self.conv_layer_2 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        self.pool_layer_2 = nn.MaxPool1d(data_length//kernel_size)
        self.fc_layer_3 = nn.Linear(128, 128)
        self.fc_layer_4 = nn.Linear(128, 9)
    
    def forward(self, input_, loss_fn, fake_label, label):
        temp = F.one_hot(input_,num_classes=257).float()
        temp.requires_grad = True
        temp.retain_grad()
        for _ in range(10):
            x = temp @ self.embedding.weight
            x = x.transpose(-1,-2)
            x_conv_1 = self.conv_layer_1(x[:,:4,:])
            x_conv_2 = torch.sigmoid(self.conv_layer_2(x[:,4:,:]))
            x = x_conv_1*x_conv_2
            del x_conv_1,x_conv_2
            x = self.pool_layer_2(x).squeeze()
            x = self.fc_layer_3(x)
            x = self.fc_layer_4(x)
            # fake_label = torch.zeros_like(x)
            print((torch.argmax(torch.softmax(x,dim=-1),dim=-1).tolist() , label.tolist()))
            loss = loss_fn(x,fake_label).cuda()
            print(loss)
            loss.backward()
            data = [(i,inpu,te,gr) for i,(inpu,te,gr) in enumerate(zip(
                input_.detach().cpu().numpy(),
                temp.detach().cpu().numpy(),
                temp.grad.detach().cpu().numpy()
            ))]
            with mp.Pool(processes=24 if len(data) > 24 else len(data)) as pool:
                results = pool.starmap(mp_func,data)
            
            check = [r[2] for r in results]
            results = sorted(results,key = lambda x: x[0])
            print([(r[2],temp.grad[i,r[2]:,:].sum(-1).sum(-1).tolist()) for i,r in enumerate(results)])
            if sum(check)/len(check) == LEAVE_BIT_NUMBER-1: break
            for i in range(len(temp)):
                temp.data[i] = torch.tensor(results[i][1], dtype=torch.float, requires_grad=True).cuda()
                
        return x.cpu().detach().numpy(),temp,check

In [12]:
# class Model(nn.Module):
#     def __init__(self, data_length = 2e6, kernel_size = 500):
#         super().__init__()
#         self.embedding = nn.Embedding(257, 8, padding_idx=0)
#         self.conv_layer_1 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
#         # self.bn_1 = nn.BatchNorm1d(128)
#         self.conv_layer_2 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
#         self.pool_layer_2 = nn.MaxPool1d(data_length//kernel_size)
#         self.fc_layer_3 = nn.Linear(128, 128)
#         self.fc_layer_4 = nn.Linear(128, 9)
    
#     def forward(self, input_, loss_fn):
#         temp = F.one_hot(input_,num_classes=257).float()
#         temp.requires_grad = True
#         temp.retain_grad()
#         for _ in range(6):
#             x = temp @ self.embedding.weight
#             x = torch.transpose(x, -1, -2)

#             x_conv_1 = self.conv_layer_1(x[:,:4,:])
#             x_conv_2 = torch.sigmoid(self.conv_layer_2(x[:,4:,:]))

#             x = x_conv_1*x_conv_2
#             del x_conv_1,x_conv_2
#             x = self.pool_layer_2(x).squeeze()
            
#             x = self.fc_layer_3(x)
#             x = self.fc_layer_4(x)
            
#             print(torch.argmax(torch.softmax(x,dim=-1),dim=-1).float().mean())
            
#             mislead_labels = torch.zeros_like(x).cuda()
#             loss = loss_fn(x,mislead_labels)
#             loss.backward()
            
#             data = [(i,inpu,te,gr) for i,(inpu,te,gr) in enumerate(zip(
#                 input_.detach().cpu().numpy(),
#                 temp.detach().cpu().numpy(),
#                 temp.grad.detach().cpu().numpy()
#             ))]
#             with mp.Pool(processes=24 if len(data) > 24 else len(data)) as pool:
#                 results = pool.starmap(mp_func,data)
#             results = sorted(results,key = lambda x: x[0])
#             for i in range(len(temp)):
#                 temp.data[i] = torch.tensor(results[i][1], dtype=torch.float, requires_grad=True).cuda()
        
#         return x.detach().cpu().numpy(),temp

In [13]:
model = Model(data_length=LEAVE_BIT_NUMBER,kernel_size=KERNEL_SIZE)

ce_loss = nn.CrossEntropyLoss()
optim = Adam(model.parameters())

model = model.cuda() if CUDA else model
ce_loss = ce_loss.cuda() if CUDA else ce_less

In [14]:
model.load_state_dict(torch.load(trained_model_path + best_trained_model))

<All keys matched successfully>

In [15]:
if __name__=='__main__':
    model.eval()
    acc = []
    preds = []
    labels = []
    for step, (batch_data,batch_label) in enumerate(tqdm(validloader)):
        batch_data = batch_data.cuda() if CUDA else batch_data
        batch_label = batch_label.cuda() if CUDA else batch_label
        batch_label = batch_label.squeeze() - 1
        fake_label = torch.zeros_like(batch_label)
        pred,temp,check = model(batch_data,ce_loss,fake_label,batch_label)
        if sum(check)/len(check) == LEAVE_BIT_NUMBER-1: continue
            
        pred = np.argmax(pred,1)
        batch_label = batch_label.cpu().data.numpy()

        preds.extend(pred.tolist())
        labels.extend(batch_label.tolist())

        count = total = 0
        for c,ans in zip(check,batch_label == pred):
            if c != LEAVE_BIT_NUMBER-1:
                count += ans
                total += 1
        acc.append(count/total)
        print(f"test：{count/total}, test mean: {np.mean(acc)}")

  0%|          | 0/725 [00:00<?, ?it/s]

([2, 2, 2], [2, 2, 2])
tensor(13.0813, device='cuda:0', grad_fn=<NllLossBackward0>)
[(499999, 0.0), (499999, 0.0), (499999, 0.0)]


  0%|          | 1/725 [00:26<5:20:14, 26.54s/it]

([1, 0, 5], [1, 0, 5])
tensor(4.4735, device='cuda:0', grad_fn=<NllLossBackward0>)
[(499999, 0.0), (442367, 0.12569290399551392), (499999, 0.0)]
([1, 7, 5], [1, 0, 5])
tensor(4.8047, device='cuda:0', grad_fn=<NllLossBackward0>)
[(499999, 0.0), (442367, 3.6421713829040527), (499999, 0.0)]
([1, 0, 5], [1, 0, 5])
tensor(4.4591, device='cuda:0', grad_fn=<NllLossBackward0>)
[(499999, 0.0), (442367, 4.033941268920898), (499999, 0.0)]
([1, 7, 5], [1, 0, 5])
tensor(5.2205, device='cuda:0', grad_fn=<NllLossBackward0>)
[(499999, 0.0), (442367, 1.241539716720581), (499999, 0.0)]
([1, 0, 5], [1, 0, 5])
tensor(4.4260, device='cuda:0', grad_fn=<NllLossBackward0>)
[(499999, 0.0), (442367, 1.343736171722412), (499999, 0.0)]
([1, 0, 5], [1, 0, 5])
tensor(4.4254, device='cuda:0', grad_fn=<NllLossBackward0>)
[(499999, 0.0), (442367, 1.4270483255386353), (499999, 0.0)]
([1, 0, 5], [1, 0, 5])
tensor(4.4251, device='cuda:0', grad_fn=<NllLossBackward0>)
[(499999, 0.0), (442367, 1.5045229196548462), (499999, 

  0%|          | 2/725 [05:05<35:08:14, 174.96s/it]

...

test：1.0, test mean: 0.2826086956521739
([2, 1, 0], [2, 1, 0])
tensor(7.2976, device='cuda:0', grad_fn=<NllLossBackward0>)
[(499999, 0.0), (499999, 0.0), (499999, 0.0)]


 18%|█▊        | 128/725 [3:52:28<26:32:09, 160.02s/it]

([2, 1, 2], [2, 1, 2])
tensor(9.0004, device='cuda:0', grad_fn=<NllLossBackward0>)


Process Process-8:
Process Process-4:
Process ForkPoolWorker-1644:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/util.py", line 296, in _exit_function
    _run_finalizers(0)
  File "/usr/lib/python3.6/multiprocessing/util.py", line 262, in _run_finalizers
    finalizer()
  File "/usr/lib/python3.6/multiprocessing/util.py", line 295, in _exit_function
    debug('running all "atexit" finalizers with priority >= 0')
  File "/usr/lib/python3.6/multiprocessing/util.py", line 48, in debug
    def debug(msg, *args):
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/util.py", line 188, in __call__
    self._kwargs = self._key = None
KeyboardInterrupt
Process Process-13:
Trac

RuntimeError: DataLoader worker (pid 8013) exited unexpectedly with exit code 1. Details are lost due to multiprocessing. Rerunning with num_workers=0 may give better error trace.

In [14]:
if __name__=='__main__':
    model.eval()
    acc = []
    preds = []
    labels = []
    for step, (batch_data,batch_label) in enumerate(tqdm(validloader)):
        batch_data = batch_data.cuda() if CUDA else batch_data
        batch_label = batch_label.cuda() if CUDA else batch_label
        batch_label = batch_label.squeeze() - 1
        fake_label = torch.zeros_like(batch_label)
        pred,temp,check = model(batch_data,ce_loss,fake_label,batch_label)
        if sum(check)/len(check) == LEAVE_BIT_NUMBER-1: continue
            
        pred = np.argmax(pred,1)
        batch_label = batch_label.cpu().data.numpy()

        preds.extend(pred.tolist())
        labels.extend(batch_label.tolist())

        count = total = 0
        for c,ans in zip(check,batch_label == pred):
            if c != LEAVE_BIT_NUMBER-1:
                count += ans
                total += 1
        acc.append(count/total)
        print(f"test：{count/total}, test mean: {np.mean(acc)}")

  0%|          | 0/725 [00:00<?, ?it/s]

([0, 0, 0], [0, 0, 0])
tensor(0.3624, device='cuda:0', grad_fn=<NllLossBackward0>)
[(499999, -0.0005131479119881988), (499999, 0.0), (499999, 0.0)]


  0%|          | 1/725 [00:25<5:10:30, 25.73s/it]

([2, 8, 0], [2, 8, 0])
tensor(6.7551, device='cuda:0', grad_fn=<NllLossBackward0>)
[(499999, 0.0), (499999, 0.0), (499999, 0.0)]


  0%|          | 2/725 [00:51<5:07:17, 25.50s/it]

...

100%|█████████▉| 723/725 [23:52:41<04:04, 122.28s/it]

([1, 7, 0], [1, 7, 1])
tensor(5.6549, device='cuda:0', grad_fn=<NllLossBackward0>)
[(499999, 0.0), (380927, 0.21022096276283264), (331775, -0.06582143902778625)]
([1, 7, 1], [1, 7, 1])
tensor(5.8740, device='cuda:0', grad_fn=<NllLossBackward0>)
[(499999, 0.0), (380927, -9.797591209411621), (331775, 13.135656356811523)]
([1, 7, 0], [1, 7, 1])
tensor(3.7770, device='cuda:0', grad_fn=<NllLossBackward0>)
[(499999, 0.0), (380927, -4.0870680809021), (331775, 14.285801887512207)]
([1, 7, 1], [1, 7, 1])
tensor(2.5144, device='cuda:0', grad_fn=<NllLossBackward0>)
[(499999, 0.0), (380927, 0.23506850004196167), (331775, 25.37211036682129)]
([1, 0, 0], [1, 7, 1])
tensor(1.7693, device='cuda:0', grad_fn=<NllLossBackward0>)
[(499999, 0.0), (380927, -0.22106504440307617), (331775, 26.410747528076172)]
([1, 0, 0], [1, 7, 1])
tensor(1.6995, device='cuda:0', grad_fn=<NllLossBackward0>)
[(499999, 0.0), (380927, -0.4983872175216675), (331775, 26.808582305908203)]
([1, 0, 0], [1, 7, 1])
tensor(1.6770, devi

100%|█████████▉| 724/725 [23:57:21<02:49, 169.46s/it]

test：0.0, test mean: 0.24414062500000003
([0, 2], [0, 2])
tensor(6.0584, device='cuda:0', grad_fn=<NllLossBackward0>)
[(499999, 0.0), (499999, 0.0)]


100%|██████████| 725/725 [23:57:41<00:00, 118.98s/it]


In [15]:
# if __name__=='__main__':
#     acc = []
#     preds = []
#     labels = []

#     for step, val_batch_data in enumerate(validloader):
#         print(step,len(validloader))
#         cur_batch_size = val_batch_data[0].size(0)

#         exe_input = val_batch_data[0].to(device) if use_gpu else val_batch_data[0]

#         label = val_batch_data[1].to(device) if use_gpu else val_batch_data[1]
#         fake_label = torch.zeros_like(label.squeeze())
#         # fake_label[fake_label == 9] = 0
#         # temp = torch.zeros((6,9))
#         # for i,j in enumerate(fake_label): temp[i][j] = 1
#         # fake_label = temp.cuda()
#         # fake_label = fake_label.reshape(-1,1)
#         label = label.squeeze() - 1
        
#         pred,temp = malconv(exe_input,ce_loss,fake_label,label)
        
#         pred = np.argmax(pred,1)
#         label = label.cpu().data.numpy().astype(int)
#         # temp_grad,temp = malconv(exe_input)
#         preds.extend(pred.tolist())
#         labels.extend(label.tolist())

#         val_Macc = (label == pred).sum()
#         val_Macc = val_Macc / cur_batch_size
#         acc.append(val_Macc)
#         print(f"test：{val_Macc}, test mean: {np.mean(acc)}")

In [16]:
temp.grad[0,123:].sum(-1).sum(-1)

tensor(3.2881, device='cuda:0')